In [15]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import re 
import datetime

%matplotlib inline

In [16]:
df = pd.read_csv('./commute_info.csv')

#Making EST and rounding off seconds
df['time'] = pd.to_datetime(df['time']) - pd.Timedelta(4, unit='h')
df['time'] = df.time.dt.round('1min')
df['date'] = df.time.dt.date
df = df.drop(df.loc[df['date'] == datetime.date(2018, 10, 31)].index).reset_index(drop=True)

In [17]:
def duration_clean(mins):
    """Eliminates text from duration descriptions and corrects hours to minutes"""
    if 'h' in mins:
        vals = re.findall(r'\d+', mins)
        return int(vals[0]) * 60 + int(vals[1])
    else:
        return int(re.search(r'\d+', mins).group())
    
df['duration_in_traffic'] = df['duration_in_traffic'].apply(duration_clean)

In [18]:
df['is_morning'] = df.time.apply(lambda x: 1 if x.hour < 11 else 0)
df['hour'] = df.time.dt.time
df['is_weekday'] = df.time.apply(lambda x: 1 if x.dayofweek < 5 else 0)

In [19]:
morning = df.loc[df['is_morning'] == 1].copy()
evening = df.loc[df['is_morning'] == 0].copy()

In [48]:
dates = list(morning['date'].unique())
traces = []

for date in dates:
    morn = morning.loc[morning['date'] == date].copy()
    color = 'blue'
    if morn['is_weekday'].max() == 0:
        color = 'red'
    trace0 = go.Scatter(x=morn['hour'], 
                        y=morn['duration_in_traffic'], 
                        name=str(date), 
                        showlegend=False,
                        line=dict(color=color, dash='dash'),
                        opacity=0.3)
    traces.append(trace0)

week_morn = morning.loc[morning['is_weekday'] == 1].groupby('hour', as_index=False)['duration_in_traffic'].mean()

trace1 = go.Scatter(x=week_morn['hour'], 
                    y=week_morn['duration_in_traffic'], 
                    name="Mean Weekday Duration", 
                    showlegend=True,
                    line=dict(color='blue',dash='dashdot', width=4.5))  

traces.append(trace1)

end_morn = morning.loc[morning['is_weekday'] == 0].groupby('hour', as_index=False)['duration_in_traffic'].mean()

trace2 = go.Scatter(x=end_morn['hour'], 
                    y=end_morn['duration_in_traffic'], 
                    name="Mean Weekend Duration", 
                    showlegend=True,
                    line=dict(color='red',dash='dashdot', width=4.5))  

traces.append(trace2)

data = traces

layout = go.Layout(title="My Morning Commute (<i>Boston to Bedford</i>)",
                   xaxis=dict(title='Time (EST)'), 
                   yaxis=dict(title='Duration of Trip (Minutes)',
                              range=[0, 70]))

fig = go.Figure(data=data, layout=layout)

py.plotly.plot(fig, filename='commute_times.html')

'https://plot.ly/~slevin886/532'

In [27]:
# dates = list(evening['date'].unique())
# traces = []

# for date in dates:
#     morn = evening.loc[evening['date'] == date].copy()
#     color = 'blue'
#     if morn['is_weekday'].max() == 0:
#         color = 'red'
#     trace0 = go.Scatter(x=morn['hour'], 
#                         y=morn['duration_in_traffic'], 
#                         name=str(date), 
#                         showlegend=False,
#                         line=dict(color=color, dash='dash'),
#                         opacity=0.4)
#     traces.append(trace0)


# data = traces

# layout = go.Layout(xaxis=dict(title='Time (EST)'), 
#                    yaxis=dict(title='Duration of Trip (Minutes)',
#                               range=[0, 70]))

# fig = go.Figure(data=data, layout=layout)

# py.offline.plot(fig, filename='commute_times.html')

'file:///Users/slevin886/Desktop/DataProjects/trip_duration/commute_times.html'

In [44]:
week_morn

,hour,duration_in_traffic
0,06:00:00,28.764706
1,06:05:00,28.764706
2,06:10:00,28.764706
3,06:15:00,28.588235
4,06:20:00,28.647059
5,06:25:00,28.647059
6,06:30:00,28.823529
7,06:35:00,28.941176
8,06:40:00,28.941176
9,06:45:00,28.941176
